# Metatables and Metamethods

Metatables allow us to change the behaviour of a table. When we apply an operation on a table, it first checks it metatable and then perform.

```lua
t = {}
print(getmetatable(t)) => nil
```

```lua
t = {}
setmetatable(t, t1)
assert(getmetatable(t) == t1) => true
```

Metatable gives opportunity to override common methods
```
The available metamethods are:
__add (LHS + RHS)
__sub (LHS - RHS)
__mul (LHS * RHS)
__div (LHS / RHS)
__mod (LHS % RHS)
__pow (LHS ^ RHS)
__unm (-RHS)
__concat (LHS .. RHS)
__len (#RHS)
__eq (LHS == RHS)
__lt (LHS < RHS)
__le (LHS <= RHS)
__index - Table access
__newindex - Table assignment
__call - Whenever the object is called like a function
__tostring - Lets you decide what using tostring() on your object will return

```

You define one table and then corresponding methods which can be overriden for example

*LOCAL VARIABLES ARE NOT VISIBLE IN INTERPRETER.

because it runs each line in different scope

```lua
local mt = {} -- if it is executed inside a file. it will be visible in whole script.

local obj = {}
setmetatable(obj,mt)
```

```lua
function mt:__tostring()
return self.r .. self.b .. self.g
end

function mt:__mul(m)
self.r = self.r*m
self.g = self.g*m
self.b = self.b*m
end

function mt:__div(m)
self.r = self.r / m
self.g = self.g / m
self.b = self.b / m
end
```

### Set manipulation library

```lua
Set = {}
    
    function Set.new (t)
      local set = {}
      for _, l in ipairs(t) do set[l] = true end
      return set
    end
    
    function Set.union (a,b)
      local res = Set.new{}
      for k in pairs(a) do res[k] = true end
      for k in pairs(b) do res[k] = true end
      return res
    end
    
    function Set.intersection (a,b)
      local res = Set.new{}
      for k in pairs(a) do
        res[k] = b[k]
      end
      return res
    end

```

We need to create a metatable and assign magic function value to above functions.

```lua
Set.mt = {}
setmetatable(set, Set.mt)
Set.mt.__add = Set.union
Set.mt.__mul = Set.intersection

```

__index method helps to search for undefined access.

if __index is assigned to a table it searches in that method.

```lua
Window.mt.__index = Window.prototype
```

When we call Window.prototype['width'] it looks for its method in current table if not foudn then it looks up in __index table.

__newindex method is used when there is an update method called.

there is a raw function that allows you to bypass the metamethod: The call rawset(t, k, v) sets the value v in key k of table t without invoking any metamethod.

## Package

Lua defines package inside a file which is nothing but table. In the end it should return one table which will be embedded in global dictionary.

Even if doesn't return it is still present in global dictionary.

Fr example if complex.lua is a file. it contains definition and returns table which will. Usually it also defines names which will be used in global table itself

For example

```lua
complex = {real=0, imaginary=0}

complex:new(d)
self.real = d.real
self.imaginary = d.imaginary
return
```
if I call require("complex.lua") it will always create complex class in global namespace. It can be overridden by _REQUIREDNAME as:

```lua
local P = {}   -- package
    if _REQUIREDNAME == nil then
      complex = P
    else
      _G[_REQUIREDNAME] = P
    end
```

## Local environment
One of the problem with the environment is that it is global. Any modification you do on it affects all parts of the program.

For package there must be a way to use only local environment.

```
a = 1
-- change current environment to empty table
setfenv(1, {})
print(a)  => nill
```

Here is nice way of doing it.
```lua

a = 1   -- create a global variable
    -- change current environment
    setfenv(1, {_G = _G})
    _G.print(a)      --> nil
    _G.print(_G.a)   --> 1
    
```

or better
```lua
a = 1
    local newgt = {}        -- create new environment
    setmetatable(newgt, {__index = _G})
    setfenv(1, newgt)    -- set it
    print(a)          --> 1
```

__index mentions next table to search for if current table doesn't have any.

In this code, the new environment inherits both print and a from the old one. Nevertheless, any assignment goes to the new table. There is no danger of changing a really global variable by mistake, although you still can change them through _G:

```lua
    -- continuing previous code
    a = 10
    print(a)      --> 10
    print(_G.a)   --> 1
    _G.a = 20
    print(_G.a)   --> 20
```

## Modules

Create an example file mymodule.lua with the following content:
```lua
local mymodule = {}

function mymodule.foo()
    print("Hello World!")
end

return mymodule
```
Now to use this new module in the interactive interpreter, just do:
```
> mymodule = require "mymodule"
> mymodule.foo()
Hello World!
```

In an actual script file, it would be recommended to make the mymodule variable local:
```
local mymodule = require "mymodule"
mymodule.foo()
```

Other ways to create module
```lua
local function private()
    print("in private function")
end

local function foo()
    print("Hello World!")
end

local function bar()
    private()
    foo() -- do not prefix function call with module
end

return {
  foo = foo,
  bar = bar,
}
```

Or we can do also like this
```lua
local mymodule = {}

local function private()
    print("in private function")
end

function mymodule.foo()
    print("Hello World!")
end

function mymodule.bar()
    private()
    mymodule.foo() -- need to prefix function call with module
end

return mymodule


```

There is another way of doing it:
Or if you don't want to have to save all the globals you need into locals:

```lua
local M = {}
do
	local globaltbl = _G
	local newenv = setmetatable({}, {
		__index = function (t, k)
			local v = t[k]
			if v == nil then return globaltbl[k] end
			return v
		end,
		__newindex = M,
	})
	if setfenv then
		setfenv(1, newenv) -- for 5.1
	else
		_ENV = newenv -- for 5.2
	end
end

local function private()
    print("in private function")
end

function foo()
    print("Hello World!")
end

function bar()
    private()
    foo()
end

return M
```